## Install python libraries

The following cell can be used to ensure that the python libraries used
in this test notebook are installed.
These may be pre-installed in future notebook images.
Once this cell has been run, it need not be re-run unless you have restarted your jupyter server.

In [1]:
# Install the library dependencies used in this notebook
# (comment this out if you prefer to not re-run this cell)
%pip install trino python-dotenv

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Loading credentials

The following cell finds a `credentials.env` file at the jupyter "home" (top level) directory.

Values in this `dotenv` file are loaded into the `os.environ` table,
as if they were regular environment variables.

Credentials are stored in `dotenv` files so that they can be referred to by standard
environment variable names, and do not appear in notebooks or other code,
which would be a security leak.

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

## Connect to trino

The following cell creates a trino api connection.

It assumes that your `credentials.env` file has been edited so that
`TRINO_PASSWD` has a JWT token obtained from:
https://das-odh-trino.apps.odh-cl1.apps.os-climate.org/

Your `TRINO_USER` value should be your github username.

In [3]:
import trino
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

## Test your trino connection

This cell shows all the catalogs visible to you.
If your trino api connection initialized correctly above,
this `show catalogs` command should always succeed.

In [4]:
cur.execute('show catalogs')
cur.fetchall()

[['jmx'],
 ['osc_datacommons_dev'],
 ['osc_datacommons_iceberg_dev'],
 ['osc_datacommons_prod'],
 ['system']]